# Blockchain

Using JSON (javascript object notation) to trasnport data because lightweight, often used for server-> webpage data movement, 'self-describing'.

- Data in name value pairs
- Data seperated by commas
- Curly brackets hold objects
- Square brackets hold arrays

eg:
{
"employees":[

    {"firstName":"John", "lastName":"Doe"},
    {"firstName":"Anna", "lastName":"Smith"},
    {"firstName":"Peter", "lastName":"Jones"}
    
]
}

Obj employees is an array with 3 objects. Each obj is a record of a person witha first and last name. 

For the __HASH__, look at various crypto methods. For storing passwords and usernames, also look at salt (crypto). Each transaction in block is usually hashed, resulting in a hash or __merkel tree__.

__object.--dict--__ is a special attribute that returns the obj as a dictionary (example below).

Hash's can be __stored__ in a python __list__. __Dependency__ is required to prevent invalidation. Taking an old block and replacing it with a new one can occur, so something must be done. This can be achieved by using the previous block's hash in the current one. The first block is called the __genesis block__. It can be uniquelly created by logic ('randomly') or manually.If content of any block prev changes, _prev-hash_ field will have a __mismatch__, thus the entire chain must be recomputed. 

It is possible to recompute, thus add a __constraint__ of some kind. New Block class attribute called __nonce__.

Blocks can be added to the chain if they are __untampered__ and the __transaction order__ is preserved (prev_hash in block to be added points to hash of last block in chain).

### Mining
All __transactions__ are __unconfirmed__. Putting the block on the chain and __computing proof of work__ is called mining. When __nonce__ satisfie our constraints, block is __mined__ and can be added to the chain. Often a successful mining operation results in a cryptocurrency __reward__.  


#### Also See: 
Merkel tree.

HashCash for proof of work.

Mining happens on all transactions in list then is cleared. What if we want only the first one to be removed once succesfully added.

In [14]:
class MyClass(object):
    class_var = 1

    def __init__(self, i_vaw, b_var):
        self.i_var = i_vaw
        self.b_var = b_var

foo = MyClass(2, 4)
bar = MyClass(3, 6)

print (MyClass.__dict__)
print (foo.__dict__)
print (bar.__dict__)


{'__module__': '__main__', 'class_var': 1, '__init__': <function MyClass.__init__ at 0x7f7c0c2c1950>, '__dict__': <attribute '__dict__' of 'MyClass' objects>, '__weakref__': <attribute '__weakref__' of 'MyClass' objects>, '__doc__': None}
{'i_var': 2, 'b_var': 4}
{'i_var': 3, 'b_var': 6}


In [79]:
from hashlib import sha256
import json
import time
import pdb   # pdb.set_trace()

class Block:
    def __init__(self, index, transac, timestamp, prev_hash):
        """ Block constructor
            :param index: unique block ID
            :param transac: list of transactions
            :param timestamp: Time generation of block
            :param previous_hash: Previous block's hash of which this is a part off """
        self.index = index
        self.transac = transac
        self.timestamp = timestamp
        self.prev_hash = prev_hash
        self.nonce = 0

    def hasher(self):
        """ Returns hash of block after convertnig
            to JSON string """
        block_json_str = json.dumps(self.__dict__, sort_keys=True)
        return sha256(block_json_str.encode()).hexdigest()
    
class Blockchain:
    #Proof of work difficulty
    difficulty = 2
    
    def __init__(self):
        """ Blockchain constructor """
        self.unconfirmed_transactions = []
        self.chain = []
        self.genesis_block()
        
    def genesis_block(self):
        """ Generates the index 0 
        block and appends it to chain """
        gb = Block(0, [], time.time(), "0")            # Block obj
        gb.hash = gb.hasher()                          # Created Attribute
        self.chain.append(gb)
        
    @property
    def last_block(self):
        """ Retrieves last block & returns an object of class 'Block'.
            Will always have at least one (genesis) """
        return self.chain[-1]
        
    def add_block(self, block, proof):
        """ Func which adds block to chain after verification
            * Checks if proof is valid.
            * Checks if prev_hash of block matches latest block. """
        previous_hash = self.last_block.hash
        
        if previous_hash != block.prev_hash:
            return False
        if not self.is_valid_proof(block, proof):
            return False
        
        block.hash = proof
        self.chain.append(block)
        return True

    def is_valid_proof(self, block, block_hash):
        """ Check if block_hash is valid & satisfies difficulty """
        return (block_hash.startswith('0'*Blockchain.difficulty) and block_hash == block.hasher())

    def proof_of_work(self, block):
        """ Tries diff vals of nonce to get 
            hash that satisfies difficulty """
        block.nonce = 0 
        
        computed_hash = block.hasher()
        while not computed_hash.startswith('0'*Blockchain.difficulty):
            block.nonce += 1
            computed_hash = block.hasher()
        
        return computed_hash
    
    def add_new_transaction(self, transac):
        self.unconfirmed_transactions.append(transac)
    
    def mine(self):
        """ Interface to add pending transactions to 
            blockchain by adding to block and figuring
            proof of work """
        if not self.unconfirmed_transactions:
            return False
        # should add a loop here to iterate through each unconfirmed transaction? 
        # or maybe have groups of transactions? 
        new_block = Block(self.last_block.index+1, self.unconfirmed_transactions, time.time(), self.last_block.hash)
        proof = self.proof_of_work(new_block)
        self.add_block(new_block, proof)
        self.unconfirmed_transactions = []
        return new_block.index

def test_hashes(current_blockchain):
    """ Func to test the hashes in the chain """
    for i in current_blockchain.chain:
        print(i.hash)
    return current_blockchain.chain[-1].hash

def get_transac(current_blockchain, searching_hash):
    """ Func to get block and its data back.
        Takes in a hash and returns the block on the chain. """
    for i in current_blockchain.chain:
        if i.hash == searching_hash:
            return i

In [80]:
new = Blockchain()

In [81]:
new.add_new_transaction(4000)
new.mine()
latest_block = test_hashes(new)

block = get_transac(new, latest_block)

print(block.transac)
print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(block.timestamp)))

e58b8c05f67d5a31a73398eb71a1b75b33ca734501c67eec767e24710a48b3d0
007a23136d88ebf808875dfe6d087e610b6d0de9b7386873ded36c3af427b86a
[4000]
2020-08-07 21:41:45


In [82]:
new.add_new_transaction(4000)
new.add_new_transaction(5000)
new.add_new_transaction(6000)
new.mine()
latest_block = test_hashes(new)

block = get_transac(new, latest_block)
print(block.transac)
print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(block.timestamp)))

e58b8c05f67d5a31a73398eb71a1b75b33ca734501c67eec767e24710a48b3d0
007a23136d88ebf808875dfe6d087e610b6d0de9b7386873ded36c3af427b86a
004e595657c4a9bbf71c68259aa7b10e165ed613a6252324a0ee45afa4f3ff16
[4000, 5000, 6000]
2020-08-07 21:41:55
